# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [53]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [54]:
weather_data = pd.read_csv('/Users/williampappas/Desktop/API/Homework/Exports/weather_df.csv')
weather_data["Latitude"] = weather_data["Latitude"].fillna(0)
weather_data["Longitude"] = weather_data["Longitude"].fillna(0)
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,0,cayenne,GF,2020-11-14 10:51:04,4.93,-52.33,100.0,66.0,86.270,3.10
1,1,sept-iles,CA,2020-11-14 10:47:15,50.20,-66.38,90.0,95.0,33.278,3.10
2,2,vaini,TO,2020-11-14 10:51:04,-21.20,-175.20,20.0,88.0,75.470,2.10
3,3,victoria,HK,2020-11-14 10:49:37,22.29,114.16,100.0,77.0,73.274,0.89
4,4,kruisfontein,ZA,2020-11-14 10:51:04,-34.00,24.73,26.0,60.0,70.268,3.13


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [55]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity (%)"]

In [56]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
funnel = weather_data.loc[(weather_data["Wind Speed (mph)"] <= 10) & (weather_data["Cloudiness (%)"] == 0) & \
                                   (weather_data["Max Temp (F)"] >= 70) & (weather_data["Max Temp (F)"] <= 80)].dropna()

funnel

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
21,21,srivardhan,IN,2020-11-14 10:51:06,18.03,73.02,0.0,75.0,79.988,3.28
42,42,tadine,NC,2020-11-14 10:51:09,-21.55,167.88,0.0,86.0,73.958,2.66
50,50,cidreira,BR,2020-11-14 10:51:10,-30.18,-50.21,0.0,77.0,74.984,4.83
68,68,pakaur,IN,2020-11-14 10:47:57,24.63,87.85,0.0,51.0,72.464,1.46
188,188,kanigiri,IN,2020-11-14 10:51:27,15.40,79.52,0.0,83.0,75.344,1.83
191,191,tekkali,IN,2020-11-14 10:51:28,18.62,84.23,0.0,68.0,75.632,3.42
200,200,wattegama,LK,2020-11-14 10:51:29,6.80,81.48,0.0,100.0,79.070,2.10
221,221,kodoli,IN,2020-11-14 10:51:32,16.88,74.20,0.0,60.0,73.940,2.29
275,275,atambua,ID,2020-11-14 10:51:40,-9.11,124.89,0.0,74.0,77.288,0.48
280,280,angoche,MZ,2020-11-14 10:51:40,-16.23,39.91,0.0,66.0,77.072,5.28


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
hotel_df = funnel.loc[:,["City","Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
21,srivardhan,IN,18.03,73.02,
42,tadine,NC,-21.55,167.88,
50,cidreira,BR,-30.18,-50.21,
68,pakaur,IN,24.63,87.85,
188,kanigiri,IN,15.40,79.52,
191,tekkali,IN,18.62,84.23,
200,wattegama,LK,6.80,81.48,
221,kodoli,IN,16.88,74.20,
275,atambua,ID,-9.11,124.89,
280,angoche,MZ,-16.23,39.91,


In [59]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [60]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

print("-------End of Search-------")

Retrieving Results for Index 21: srivardhan.
Closest hotel in srivardhan is Tranquil Beach Resort.
------------
Retrieving Results for Index 42: tadine.
Missing field/result... skipping.
------------
Retrieving Results for Index 50: cidreira.
Closest hotel in cidreira is Hotel Castelo.
------------
Retrieving Results for Index 68: pakaur.
Closest hotel in pakaur is Hotel Muskan Palace.
------------
Retrieving Results for Index 188: kanigiri.
Missing field/result... skipping.
------------
Retrieving Results for Index 191: tekkali.
Missing field/result... skipping.
------------
Retrieving Results for Index 200: wattegama.
Missing field/result... skipping.
------------
Retrieving Results for Index 221: kodoli.
Missing field/result... skipping.
------------
Retrieving Results for Index 275: atambua.
Closest hotel in atambua is Hotel King Star.
------------
Retrieving Results for Index 280: angoche.
Closest hotel in angoche is Guest House HESADA.
------------
Retrieving Results for Index 29

In [61]:
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
21,srivardhan,IN,18.03,73.02,Tranquil Beach Resort
42,tadine,NC,-21.55,167.88,
50,cidreira,BR,-30.18,-50.21,Hotel Castelo
68,pakaur,IN,24.63,87.85,Hotel Muskan Palace
188,kanigiri,IN,15.40,79.52,


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [63]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))